In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
import numpy as np

In [22]:
file_path = r"C:\Users\raaja\OneDrive\Documents\maichu development\notebooks\merged_data_with_labels.csv"
df = pd.read_csv(file_path)

classifier_columns = ["windspeedKmph", "winddirDegree", "precipMM", "visibility", "pressure", 
                      "cloudcover", "DewPointF", "WindGustKmph", "tempF", "WindChillF", "humidity", "DepDelayMinutes", "ArrDelayMinutes"]


In [23]:
X_train_temp_clf, X_test_temp_clf, y_train_clf, y_test_clf = train_test_split(df[classifier_columns], df["IsDelayed"], test_size=0.2, random_state=42)
X_train_clf = X_train_temp_clf.drop("ArrDelayMinutes", axis=1)
X_test_clf = X_test_temp_clf.drop("ArrDelayMinutes", axis=1)

In [24]:
classifier = DecisionTreeClassifier(criterion="gini", random_state=42, max_depth=8, min_samples_leaf=5)
classifier.fit(X_train_clf, y_train_clf)

DecisionTreeClassifier(max_depth=8, min_samples_leaf=5, random_state=42)

In [25]:
y_pred_clf = classifier.predict(X_test_clf)

In [26]:
delayed_flights = df[df["IsDelayed"] == "Delayed"]

In [27]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(delayed_flights[classifier_columns], delayed_flights["ArrDelayMinutes"], test_size=0.2, random_state=42)
X_train_reg = X_train_reg.drop("ArrDelayMinutes", axis=1)
X_test_reg = X_test_reg.drop("ArrDelayMinutes", axis=1)

In [28]:
regressor = GradientBoostingRegressor()
regressor.fit(X_train_reg, y_train_reg)



GradientBoostingRegressor()

In [29]:
delayed_flights_temp = X_test_temp_clf[y_pred_clf == "Delayed"]

target_column = "ArrDelayMinutes"
columns_to_drop = [col for col in delayed_flights_temp if col != target_column]
y_accuracy_test = delayed_flights_temp.drop(columns_to_drop, axis=1)
delayed_flights_test = delayed_flights_temp.drop(target_column, axis=1)

In [30]:
delayed_flights_test

,windspeedKmph,winddirDegree,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,DepDelayMinutes
1079740,18,175,0.0,8,1010,68,63,23,66,66,90,76.0
1093872,16,226,0.0,10,1018,42,53,18,60,60,79,41.0
550997,9,84,3.3,9,1016,100,62,13,65,64,94,12.0
1758437,9,259,0.0,10,1014,41,66,10,69,69,88,30.0
639641,12,264,0.3,10,1009,85,60,19,79,77,61,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...
641295,11,181,0.2,10,1010,16,63,12,82,83,53,303.0
274636,19,149,0.1,6,1018,75,69,23,73,72,90,22.0
1535529,16,5,0.5,10,1021,56,34,27,36,30,85,47.0
964432,9,105,0.0,10,1040,1,22,12,31,24,72,19.0


In [31]:
y_pred_reg = regressor.predict(delayed_flights_test)


In [32]:
y_pred_reg


array([71.15113854, 34.7255081 , 15.00459184, ..., 41.47767587,
       16.6918599 , 24.57807389])

In [36]:
import os
import csv
from sklearn.metrics import mean_squared_error, r2_score
r2 = r2_score(y_accuracy_test, y_pred_reg)
print(r2)

0.9433867834202004
